# DZ-2010-DBMML

Replicating Dowe & Nayyar 2010 *"Database Normalisation as a By-Product of Minimum Message Length Inferencing"*

In [1]:
from mml import *
import pandas as pd

## Table 8 replication

$L = 11$ in 1NF, shown in Table 8

In [2]:
t8_results = pd.DataFrame(
    [test_1nf(), test_2nf(), test_3nf()],
    columns=["name", "attr", "tables", "1P", "2P", "ML"],
)
t8_results.style.set_caption('Recreation of Table 8')
t8_results[["name", "1P", "2P", "ML"]].to_csv('output/table8.csv')
t8_results[["name", "1P", "2P", "ML"]]

,name,1P,2P,ML
0,DZ-1NF,10.228819,203.034011,213.262830
1,DZ-2NF,36.471091,154.848946,191.320037
2,DZ-3NF,46.284872,153.848946,200.133818


## Table 9 replication

In [3]:
t9_attributes_instances = [100, 100, 100, 100, 30, 30, 15, 15, 3, 3]
t9_L = 300
t9_results = pd.DataFrame(
    [
        test_1nf(t9_attributes_instances, t9_L),
        test_2nf(t9_attributes_instances, t9_L),
        test_3nf(t9_attributes_instances, t9_L),
    ],
    columns=["name", "attr", "tables", "1P", "2P", "ML"],
)
t9_results.style.set_caption("Recreation of Table 9")
t9_results[["name", "1P", "2P", "ML"]].to_csv("output/table9.csv")
t9_results[["name", "1P", "2P", "ML"]]


,name,1P,2P,ML
0,DZ-1NF,10.228819,14211.873643,14222.102462
1,DZ-2NF,36.471091,8149.535567,8186.006658
2,DZ-3NF,46.284872,7876.053225,7922.338097
